<a href="https://colab.research.google.com/github/jrmaza/machine-learning/blob/main/py3Dmol_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

py3Dmol
=======

py3DMol es un artilugio para utilizar un módulo javascript (3Dmol.js) en ```python notebooks``` con [IPython/Jupyter](http://jupyter.org/).
No se necesita un kernel IPython para usarse.

Instalación
------------


In [ ]:
!pip install py3Dmol

Invocar el módulo
------------

In [3]:
import py3Dmol

```query``` & ```setStyle```
------------
Con ```query``` podemos acceder a estructuras cristalográficas de macro-estructuras tipo proteínas con el código PDB. La subclase ```setStyle``` permite modificar el estilo de visualización de las moléculas.

La prteasa del VIH depositada en el servidor https://www.rcsb.org/structure/2Q5K, puede ser visualizada con ```py3Dmol```

In [ ]:
protease = py3Dmol.view(query='mmtf:2Q5K')
protease.setStyle({'cartoon': {'color':'spectrum'}})
protease.show()
# **** Intentar ****
# A. Cambie el atributo de tipo de representación de la molécula: 'line', 'sticks', 'sphere'
# B. Cambie el atributo de 'color' de representación de la molécula: 'pink', 'blue'

Integración con RDKit
=====================



### Install RDKit

In [ ]:
!pip install rdkit

In [6]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
rdkit.Chem.Draw.IPythonConsole.ipython_3d = True  # Activa la visualización en línea py3Dmol

### Load a Molecule

In [ ]:
mol = Chem.MolFromSmiles('Cn1cnc2c1c(=O)n(c(=O)n2C)C')
mol = Chem.rdmolops.AddHs(mol)
AllChem.EmbedMolecule(mol)
mol

## Ejemplos de visualizaciones

Vibraciones moleculares
------
Construiremos la molécula del amoniaco NH3 y asignaremos una vibración

In [24]:
xyz = '''4
* (null), Energy   -1000.0000000
N     0.000005    0.019779   -0.000003   -0.157114    0.000052   -0.012746
H     0.931955   -0.364989    0.000003    1.507100   -0.601158   -0.004108
H    -0.465975   -0.364992    0.807088    0.283368    0.257996   -0.583024
H    -0.465979   -0.364991   -0.807088    0.392764    0.342436    0.764260
'''

In [25]:
xyzview = py3Dmol.view(width=400,height=400)
xyzview.addModel(xyz,'xyz',{'vibrate': {'frames':20,'amplitude':1}})
xyzview.setStyle({'stick':{}})
xyzview.setBackgroundColor('gray')
xyzview.animate({'loop': 'backAndForth'})
xyzview.zoomTo()
xyzview.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Archivos ```mol```
========

Al igual que los archivos XYZ, también se puede cargar un archivo de extensión ```mol```

In [ ]:
benceno='''
     RDKit          3D

  6  6  0  0  0  0  0  0  0  0999 V2000
   -0.9517    0.7811   -0.6622 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.2847    1.3329   -0.3121 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.2365    0.5518    0.3512 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.9517   -0.7811    0.6644 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.2847   -1.3329    0.3144 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.2365   -0.5518   -0.3489 C   0  0  0  0  0  0  0  0  0  0  0  0
  1  2  2  0
  2  3  1  0
  3  4  2  0
  4  5  1  0
  5  6  2  0
  6  1  1  0
M  END
$$$$'''
view = py3Dmol.view(data=benceno,style={'stick':{'colorscheme':'cyanCarbon'}})
view.show()

Rejillas de representaciones
======
Se puede crear una rejilla de n-filas por m-columnas y en cada una tener una representación de diferentes moléculas.

In [ ]:
view = py3Dmol.view(query='pdb:1dc9',linked=False,viewergrid=(2,2))
view.setViewStyle({'style':'outline','color':'black','width':0.1})
view.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}},viewer=(0,1))
view.setStyle({'stick':{'colorscheme':'greenCarbon'}},viewer=(1,0))
view.setStyle({'cartoon':{'color':'spectrum'}},viewer=(1,1))
view.removeAllModels(viewer=(0,0))
view.addModel(benceno,'sdf',viewer=(0,0))
view.setStyle({'stick':{}},viewer=(0,0))
view.zoomTo(viewer=(0,0))
view.render()

Mismo objeto, diferentes representaciones
=====

Se puede crear una rejilla de n-filas por m-columnas y especificar un estilo diferente para cada una de las representaciones en el contenedor nxm

In [ ]:
view = py3Dmol.view(query='pdb:1dc9',viewergrid=(2,2),style=[[{'stick':{}},{'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}}],
                                                            [{'stick':{'colorscheme':'greenCarbon'}},{'cartoon':{'color':'spectrum'}}]])
view.show()

Diferentes representaciones
======


In [ ]:
view = py3Dmol.view(query='pdb:1ycr')
chA = {'chain':'A'}
chB = {'chain':'B'}
view.setStyle(chA,{'cartoon': {'color':'spectrum'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.7,'color':'white'}, chA)
view.setStyle(chB,{'stick':{}})
view.show()

Diferentes aplicaciones con py3Dmol
======

In [29]:
def MolTo3DView(mol, size=(300, 300), style="stick", surface=False, opacity=0.5):
    """Draw molecule in 3D
    
    Args:
    ----
        mol: rdMol, molecule to show
        size: tuple(int, int), canvas size
        style: str, type of drawing molecule
               style can be 'line', 'stick', 'sphere', 'carton'
        surface, bool, display SAS
        opacity, float, opacity of surface, range 0.0-1.0
    Return:
    ----
        viewer: py3Dmol.view, a class for constructing embedded 3Dmol.js views in ipython notebooks.
    """
    assert style in ('line', 'stick', 'sphere', 'carton')
    mblock = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view(width=size[0], height=size[1])
    viewer.addModel(mblock, 'mol')
    viewer.setStyle({style:{}})
    if surface:
        viewer.addSurface(py3Dmol.SAS, {'opacity': opacity})
    viewer.zoomTo()
    return viewer

In [ ]:
from ipywidgets import interact,fixed,IntSlider
import ipywidgets

@interact
def smi2viewer(smi='CC=O'):
    try:
        conf = smi2conf(smi)
        return MolTo3DView(conf).show()
    except:
        return None

In [ ]:
from ipywidgets import interact,fixed,IntSlider
import ipywidgets

def smi2conf(smiles):
    '''Convert SMILES to rdkit.Mol with 3D coordinates'''
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol)
        AllChem.MMFFOptimizeMolecule(mol, maxIters=200)
        return mol
    else:
        return None


smi = 'COc3nc(OCc2ccc(C#N)c(c1ccc(C(=O)O)cc1)c2P(=O)(O)O)ccc3C[NH2+]CC(I)NC(=O)C(F)(Cl)Br'
conf = smi2conf(smi)
viewer = MolTo3DView(conf, size=(600, 300), style='sphere')
viewer.show()